[**GoogLeNet (Szegedy et al., 2015)**](https://arxiv.org/abs/1409.4842) is an architecture that uses networks with multi-branch convolutions, introducing the Inception module to capture features at multiple scales efficiently.

![](./imgs/googlenet.png)

![](./imgs/inception_v1.png)

GoogLeNet was designed to improve accuracy while keeping computational cost manageable. It won the ILSVRC-2014 competition with state-of-the-art performance, surpassing AlexNet and VGG by using the Inception module, which allows the network to extract multi-scale features efficiently.

Unlike traditional CNNs, GoogLeNet processes different receptive fields (1×1, 3×3, 5×5 convolutions, and pooling layers) in parallel within each Inception module, significantly improving feature extraction. The architecture is 22 layers deep (excluding pooling layers) but remains computationally efficient due to 1×1 convolutions, which reduce the number of parameters by acting as "bottleneck" layers. Instead of fully connected layers, GoogLeNet uses Global Average Pooling (GAP) to minimize overfitting and reduce computational cost.

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import utils

In [3]:
class Inception(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):
        """ Inception module with four parallel branches. """
        super().__init__()

        # Branch 1: 1x1 conv
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels, ch1x1, kernel_size=1),
            nn.ReLU(inplace=True)
        )

        # Branch 2: 1x1 conv -> 3x3 conv
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, ch3x3red, kernel_size=1), nn.ReLU(inplace=True),
            nn.Conv2d(ch3x3red, ch3x3, kernel_size=3, padding=1), nn.ReLU(inplace=True)
        )

        # Branch 3: 1x1 conv -> 5x5 conv
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, ch5x5red, kernel_size=1), nn.ReLU(inplace=True),
            nn.Conv2d(ch5x5red, ch5x5, kernel_size=5, padding=2), nn.ReLU(inplace=True)
        )

        # Branch 4: 3x3 max pool -> 1x1 conv
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, pool_proj, kernel_size=1), nn.ReLU(inplace=True)
        )

    def forward(self, x):
        outputs = [self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)]
        return torch.cat(outputs, 1)

In [5]:
class AuxiliaryClassifier(nn.Module):
    def __init__(self, in_channels, num_classes):
        """ Auxiliary classifier used during training to combat vanishing gradients. """
        super().__init__()

        self.avgpool = nn.AvgPool2d(kernel_size=5, stride=3)
        self.conv = nn.Conv2d(in_channels, 128, kernel_size=1)
        self.fc1 = nn.Linear(128 * 4 * 4, 1024)  # Assumes 14x14 input size after pooling
        self.fc2 = nn.Linear(1024, num_classes)
        self.dropout = nn.Dropout(p=0.7)

    def forward(self, x):
        x = self.avgpool(x)
        x = F.relu(self.conv(x), inplace=True)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x), inplace=True)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [7]:
class GoogLeNet(nn.Module):
    def __init__(self, num_classes=10):
        """ GoogLeNet architecture with 9 Inception modules and 2 auxiliary classifiers. """
        super().__init__()

        # Initial conv and pool layers
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=1)
        self.conv3 = nn.Conv2d(64, 192, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Inception modules (parameters from the original paper)
        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)          # Output: 256
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)        # Output: 480
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)         # Output: 512
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)        # Output: 512
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)        # Output: 512
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)        # Output: 528
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)      # Output: 832
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)      # Output: 832
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)      # Output: 1024
        
        # Auxiliary classifiers
        self.aux1 = AuxiliaryClassifier(512, num_classes)  # After inception4a
        self.aux2 = AuxiliaryClassifier(528, num_classes)  # After inception4d
        
        # Main classifier
        self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.dropout = nn.Dropout(p=0.4)
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x, training=False):
        # Initial layers
        x = F.relu(self.conv1(x), inplace=True)
        x = self.maxpool1(x)
        x = F.relu(self.conv2(x), inplace=True)
        x = F.relu(self.conv3(x), inplace=True)
        x = self.maxpool2(x)
        
        # Inception modules
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        x = self.inception4a(x)
        if training:
            aux1 = self.aux1(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        if training:
            aux2 = self.aux2(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)
        x = self.inception5a(x)
        x = self.inception5b(x)
        
        # Main classifier
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(x)
        x = self.fc(x)
        
        if training:
            return x, aux1, aux2
        return x

In [9]:
utils.layer_summary(GoogLeNet(), (1, 3, 224, 224))

Layer Name                     Layer Type              Param #         Output Shape
conv1                          Conv2d                     9472    (1, 64, 112, 112)
maxpool1                       MaxPool2d                     0      (1, 64, 56, 56)
conv2                          Conv2d                     4160      (1, 64, 56, 56)
conv3                          Conv2d                   110784     (1, 192, 56, 56)
maxpool2                       MaxPool2d                     0     (1, 192, 28, 28)
inception3a.branch1.0          Conv2d                    12352      (1, 64, 28, 28)
inception3a.branch1.1          ReLU                          0      (1, 64, 28, 28)
inception3a.branch2.0          Conv2d                    18528                Error
inception3a.branch2.1          ReLU                          0      (1, 64, 28, 28)
inception3a.branch2.2          Conv2d                   110720                Error
inception3a.branch2.3          ReLU                          0      (1, 64, 

In [12]:
def train_step(
    train_loader: torch.utils.data.DataLoader,
    model: torch.nn.Module,
    criterion: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    device: torch.device
):
    """
    Performs one full training pass (epoch) over the training dataset for GoogLeNet.

    Args:
        train_loader (DataLoader): DataLoader for the training dataset.
        model (nn.Module): GoogLeNet model with auxiliary classifiers.
        criterion (nn.Module): Loss function (e.g., CrossEntropyLoss).
        optimizer (Optimizer): Optimizer for updating model weights.
        device (torch.device): Device to perform computations on.

    Returns:
        tuple: Average training loss and accuracy (based on main output).
    """
    model.train()
    running_loss, running_corrects = 0.0, 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        
        # Forward pass: GoogLeNet returns (main_output, aux1_output, aux2_output) during training
        main_output, aux1_output, aux2_output = model(inputs, training=True)
        
        # Compute losses for main and auxiliary outputs
        loss_main = criterion(main_output, labels)
        loss_aux1 = criterion(aux1_output, labels)
        loss_aux2 = criterion(aux2_output, labels)
        
        # Combine losses with weights (as per original GoogLeNet paper: 0.3 for auxiliaries)
        loss = loss_main + 0.3 * loss_aux1 + 0.3 * loss_aux2
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Track loss and accuracy (using main output only for accuracy)
        running_loss += loss.item()
        running_corrects += (main_output.argmax(dim=1) == labels).sum().item()

    avg_loss = running_loss / len(train_loader)
    avg_accuracy = running_corrects / len(train_loader.dataset)
    return avg_loss, avg_accuracy

In [13]:
data = utils.CIFAR10DataLoader(batch_size=64, resize=(224, 224))
train_loader = data.get_train_loader()
test_loader = data.get_test_loader()

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GoogLeNet(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
for epoch in range(epochs):
    train_loss, train_acc = train_step(train_loader, model, criterion, optimizer, device)
    test_loss, test_acc = utils.eval_step(test_loader, model, criterion, device)
    print(f"Epoch {epoch + 1}/{epochs}: Train Loss={train_loss}, Test Loss={test_loss}, Test Accuracy={test_acc}")

Epoch 1/10: Train Loss=3.199247202604933, Test Loss=1.6956925042875253, Test Accuracy=0.3624
Epoch 2/10: Train Loss=2.576447015833062, Test Loss=1.4247882161170813, Test Accuracy=0.4755
Epoch 3/10: Train Loss=2.176050790435518, Test Loss=1.2172544473295759, Test Accuracy=0.5566
Epoch 4/10: Train Loss=1.8714020403144915, Test Loss=1.0979493567898015, Test Accuracy=0.6043
Epoch 5/10: Train Loss=1.6747948440444438, Test Loss=0.9922284590210885, Test Accuracy=0.6424
Epoch 6/10: Train Loss=1.4990106914049524, Test Loss=0.8832556571170782, Test Accuracy=0.6848
Epoch 7/10: Train Loss=1.3667050271540346, Test Loss=0.8922392376668894, Test Accuracy=0.6818
Epoch 8/10: Train Loss=1.2496317306442943, Test Loss=0.7760469059275973, Test Accuracy=0.7251
Epoch 9/10: Train Loss=1.156023314160764, Test Loss=0.7694112630027115, Test Accuracy=0.7294
Epoch 10/10: Train Loss=1.0627327274026164, Test Loss=0.7275514950038521, Test Accuracy=0.7456
